In [1]:
# from astroquery.sdss import SDSS
# from astropy import coordinates as coords
# co = coords.SkyCoord('0h8m05.63s +14d50m23.3s')
# result = SDSS.query_region(co)
# print(result[:5])
import pandas as pd

In [2]:
import SciServer.CasJobs as CasJobs # query with CasJobs, the primary database for the SDSS
import SciServer.SkyServer as SkyServer # show individual objects through SkyServer
import SciServer.SciDrive
import warnings
warnings.filterwarnings('ignore')

In [15]:
testQuery = "SELECT TOP 100 \
 p.objID, p.ra, p.dec, \
 p.modelMag_r, \
 p.modelMag_u - p.modelMag_g as u_g, \
 p.modelMag_g - p.modelMag_z as g_z, \
 p.modelMag_g - p.modelMag_r as g_r, \
 p.modelMag_g - p.modelMag_i as g_i, \
 p.modelMag_r - p.modelMag_i as r_i, \
 p.modelMag_r - p.modelMag_z as r_z, \
 p.petroRad_r, p.flags, dbo.fPhotoFlagsN(p.flags) as flag_text \
FROM\
 photoObj as p \
JOIN dbo.fGetNearbyObjEq(49.9467, 41.5131, 45) as N ON N.objID = p.objID \
WHERE\
 p.modelMag_r > -14.3*(p.modelMag_g - p.modelMag_z) + 37 \
 and p.modelMag_r > -14.3*(p.modelMag_g - p.modelMag_z) + 46 \
 and p.type = 3\
 and p.flags & (dbo.fPhotoFlags('MANYPETRO')\
 + dbo.fPhotoFlags('TOO_FEW_GOOD_DETECTIONS ')\
 + dbo.fPhotoFlags('BAD_MOVING_FIT_CHILD ')\
 + dbo.fPhotoFlags('PEAKS_TOO_CLOSE ')\
 + dbo.fPhotoFlags('NODEBLEND_MOVING')) = 0"
testQuery

"SELECT TOP 100  p.objID, p.ra, p.dec,  p.modelMag_r,  p.modelMag_u - p.modelMag_g as u_g,  p.modelMag_g - p.modelMag_z as g_z,  p.modelMag_g - p.modelMag_r as g_r,  p.modelMag_g - p.modelMag_i as g_i,  p.modelMag_r - p.modelMag_i as r_i,  p.modelMag_r - p.modelMag_z as r_z,  p.petroRad_r, p.flags, dbo.fPhotoFlagsN(p.flags) as flag_text FROM photoObj as p JOIN dbo.fGetNearbyObjEq(49.9467, 41.5131, 45) as N ON N.objID = p.objID WHERE p.modelMag_r > -14.3*(p.modelMag_g - p.modelMag_z) + 37  and p.modelMag_r > -14.3*(p.modelMag_g - p.modelMag_z) + 46  and p.type = 3 and p.flags & (dbo.fPhotoFlags('MANYPETRO') + dbo.fPhotoFlags('TOO_FEW_GOOD_DETECTIONS ') + dbo.fPhotoFlags('BAD_MOVING_FIT_CHILD ') + dbo.fPhotoFlags('PEAKS_TOO_CLOSE ') + dbo.fPhotoFlags('NODEBLEND_MOVING')) = 0"

In [16]:
df0 = CasJobs.executeQuery(testQuery, "dr16")
df0

,objID,ra,dec,modelMag_r,u_g,g_z,g_r,g_i,r_i,r_z,petroRad_r,flags,flag_text
0,1237661055281594829,49.269779,41.566827,19.95320,1.836390,2.127121,1.187225,1.802418,0.615192,0.939896,2.309841,175990848487440,PSF_FLUX_INTERP DEBLENDED_AT_EDGE STATIONARY B...
1,1237661055281594893,49.284477,41.571282,22.26457,0.047655,3.104683,2.030626,2.682510,0.651884,1.074057,1.212256,246359626219792,PSF_FLUX_INTERP DEBLEND_NOPEAK DEBLENDED_AT_ED...
2,1237661055281595057,49.296095,41.533306,21.66178,1.109602,1.752148,1.067318,1.684628,0.617310,0.684830,7.358118,35184640659800,DEBLENDED_AT_EDGE BINNED1 INTERP COSMIC_RAY NO...
3,1237661055281594866,49.264991,41.539871,20.60325,2.330906,2.299870,1.194206,1.832098,0.637892,1.105663,1.278763,105622104178960,DEBLEND_NOPEAK DEBLENDED_AT_EDGE STATIONARY BI...
4,1237661055281595694,49.173305,41.565740,18.75458,1.829176,2.217201,1.263025,1.815594,0.552568,0.954176,3.011584,105622104310032,DEBLEND_NOPEAK DEBLENDED_AT_EDGE STATIONARY BI...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1237661060111402976,50.788990,41.729623,20.99666,1.114576,2.426470,1.459702,2.011494,0.551792,0.966768,1.270226,68987912192,STATIONARY BINNED1
96,1237661060111403043,50.803780,41.724878,22.22645,-0.347996,3.167782,1.678856,2.971252,1.292397,1.488926,7.359514,35255507484944,DEBLENDED_AT_EDGE STATIONARY MOVED BINNED1 NOP...
97,1237661060111403119,50.775797,41.654968,20.48092,1.375107,1.953083,1.306826,1.651907,0.345081,0.646257,3.233951,68987912448,STATIONARY BINNED1 NOPETRO
98,1237661060111403170,50.796054,41.651078,21.29469,1.659788,2.434076,1.363443,1.885487,0.522043,1.070633,1.343044,68987912448,STATIONARY BINNED1 NOPETRO


In [14]:
# df_phot = pd.DataFrame()
# for index,query in enumerate(querylist):
#     print('Executing query', index, 'of', len(atlas3d)-1)
#     df = CasJobs.executeQuery(query, "dr16")
#     df = df.assign(host_galaxy = atlas3d['galaxy'][index], host_distance= atlas3d['distance'][index], host_ra = atlas3d['RA'][index], host_dec = atlas3d['DEC'][index])
#     data = [df_phot, df]
#     df_phot = pd.concat(data, ignore_index = True, sort=False)

In [13]:
coords = df0[['ra', 'dec']]
coords.to_csv('temp.csv')